In [15]:
import pandas as pd
import sys, pathlib
sys.path.insert(0, str(pathlib.Path("..").resolve()))
from app.services.db_service import DatabaseService
from app.services.auth_service  import AuthService

In [ ]:
df = pd.read_csv('../resources/data/uploads/employee_data/employees.csv')

In [ ]:
df

In [ ]:
db_service = DatabaseService(db_path="../resources/data/db/rbac_chatbot.db")
with db_service.get_db() as conn:
    df.to_sql('employees', conn, if_exists='replace', index=False)

In [ ]:
df = pd.read_csv('../resources/data/uploads/employee_data/users.csv')

In [ ]:
db_service = DatabaseService(db_path="../resources/data/db/rbac_chatbot.db")
with db_service.get_db() as conn:
    df.to_sql('users', conn, if_exists='replace', index=False)

In [ ]:
db_service = DatabaseService(db_path="../resources/data/db/rbac_chatbot.db")
with db_service.get_db() as conn:
    df.to_sql('employees', conn, if_exists='replace', index=False)

In [ ]:
db_service = DatabaseService(db_path="../resources/data/db/rbac_chatbot.db")
with db_service.get_db() as conn:
    rows = conn.execute("SELECT * FROM users").fetchall()
    for row in rows:
        print(row)



In [ ]:
db_service = DatabaseService(db_path="../resources/data/db/rbac_chatbot.db")
auth_service = AuthService(db_service=db_service)
user = auth_service.authenticate("allison_hill", "allison123")
print(user)

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

In [4]:
markdown_loader = DoclingLoader(file_path="../resources/data/uploads/engineering/engineering_master_doc.md"
                                ,export_type=ExportType.MARKDOWN)
documents = markdown_loader.load()
documents[0].page_content


2026-01-02 13:07:35,927 - INFO - detected formats: [<InputFormat.MD: 'md'>]
2026-01-02 13:07:35,928 - INFO - Going to convert document batch...
2026-01-02 13:07:35,929 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2026-01-02 13:07:35,961 - INFO - Loading plugin 'docling_defaults'
2026-01-02 13:07:35,965 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2026-01-02 13:07:35,965 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-02 13:07:35,966 - INFO - Processing document engineering_master_doc.md
2026-01-02 13:07:45,417 - INFO - Finished converting document engineering_master_doc.md in 9.49 sec.


'# FinSolve Technologies Engineering Document\n\n## 1. Introduction\n\n### 1.1 Company Overview\n\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.\n\n### 1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalable, and innovative technology solutions."\n\n### 1.3 Scope\n\nThis document covers:\n\n- System architecture and infrastructure\n- Software development lifec

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter,MarkdownTextSplitter,MarkdownHeaderTextSplitter
from langchain_core.documents import Document

c:\Users\ravis\AI Usecases\rbac-chatbot\rbac-chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
markdown_header_text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("##", "section")])

markdown_text_splitter = MarkdownTextSplitter(chunk_size=1000,
            chunk_overlap=200,length_function=len)
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
            chunk_overlap=200,length_function=len)

In [5]:
header_docs: list[Document] = []

for doc in documents:
            splits = markdown_header_text_splitter.split_text(doc.page_content)

            for split in splits:
                # Preserve original metadata
                section = split.metadata.get("section")

                content = split.page_content.strip()

                if section:
                    content = f"## {section}\n\n{content}"

                header_docs.append(Document(page_content=content,
                                            metadata=split.metadata))

chunks = markdown_text_splitter.split_documents(header_docs)


In [1]:
header_docs[1].page_content

NameError: name 'header_docs' is not defined

In [6]:
for chunk in chunks:
            
            chunk.metadata = {
                "role": "engineering",     
                **chunk.metadata 
            }

In [8]:
chunks

[Document(metadata={'role': 'engineering'}, page_content='# FinSolve Technologies Engineering Document'),
 Document(metadata={'role': 'engineering', 'section': '1. Introduction'}, page_content='## 1. Introduction'),
 Document(metadata={'role': 'engineering', 'section': '1. Introduction'}, page_content='### 1.1 Company Overview  \nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.  \n### 1.2 Purpose  \nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment wi

In [24]:
[chunk for chunk in chunks if chunk.metadata and chunk.metadata['section'] == '2. System Architecture']

[Document(metadata={'section': '2. System Architecture'}, page_content="## 2. System Architecture\n\n### 2.1 Overview  \nFinSolve's architecture is a microservices-based, cloud-native system designed for scalability, resilience, and security. It leverages a modular design to support rapid feature development and seamless integration with third-party financial systems (e.g., payment gateways, credit bureaus, regulatory reporting systems).  \n### 2.2 High-Level Architecture"),
 Document(metadata={'section': '2. System Architecture'}, page_content='```\n[Client Apps]\n├── Mobile Apps (iOS, Android)\n├── Web App (React)\n└── APIs (REST, GraphQL)\n\n[API Gateway]\n└── AWS API Gateway (Routing, Authentication, Rate Limiting)\n\n[Microservices Layer]\n├── Authentication Service (OAuth 2.0, JWT)\n├── Payment Processing Service\n├── Wealth Management Service\n├── Analytics Service\n└── Notification Service\n\n[Data Layer]\n├── PostgreSQL (Transactional Data)\n├── MongoDB (User Profiles, Metadat

In [6]:
from qdrant_client.models import Filter, FieldCondition, MatchValue
from qdrant_client import QdrantClient

from langchain_qdrant import QdrantVectorStore
import sys, pathlib
sys.path.insert(0, str(pathlib.Path("..").resolve()))
from app.config.config import get_settings

settings = get_settings()
client = QdrantClient(
        url=settings.qdrant_url,
        api_key=settings.qdrant_api_key,
    )



In [ ]:
client.delete_payload_index(collection_name=settings.collection_name,field_name="role")


In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
        model=settings.embedding_model
    )

# vector_store = QdrantVectorStore(
#             client=client,
#             collection_name=settings.collection_name,
#             embedding=embeddings
#         )

In [ ]:
allowed_roles = list(set(['engineering'] + ["general"]))

qdrant_filter = {
    "role": {"$in": allowed_roles}
}
docs = vector_store.similarity_search_with_score(
    query="Deployment and DevOps Practices",
    k=5,
    filter=qdrant_filter
)

In [ ]:
[doc for doc in docs if doc[0].metadata['role'] in (allowed_roles)]

In [ ]:
from qdrant_client.conversions.common_types import Filter
from qdrant_client.models import Match

allowed_roles = list(set(['hr'] + ["general"]))

# qdrant_filter = Filter(
#             must=[
#                 FieldCondition(
#                     key="role",
#                     match=MatchValue(value=role)
#                 )
#                 for role in allowed_roles
#             ]
#         )

query = "Deployment and DevOps Practices"

query_vector = embeddings.embed_query(query)

results = client.query_points(
    collection_name="RBAC_CHATBOT_COLLECTION",
    query=query_vector,
    #query_vector=query_vector,
    limit=5,
    query_filter = Filter(
            must=[
                FieldCondition(
                    key="role",
                    range=Match(value="engineering")
                )
                #for role in allowed_roles
            ]
        )
)

In [ ]:
results

In [35]:
from qdrant_client.http  import models
allowed_roles = list(set(['hr'] + ["general"]))

qdrant_filter = models.Filter(
            should=[
                models.FieldCondition(
                    key="role",
                    match=models.MatchValue(value="engineering")
                )
            ]
        )

retriever = vector_store.similarity_search(
    query="Deployment and DevOps Practices",
    k=5,
    filter = qdrant_filter

)


NameError: name 'vector_store' is not defined

In [ ]:
retriever

In [ ]:
docs = retriever.invoke("Deployment and DevOps Practices")


In [ ]:
docs

In [ ]:
points, _ = client.scroll(
    collection_name="RBAC_CHATBOT_COLLECTION",
    limit=1,
    with_payload=True
)

print(points[0])


In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, Filter, FieldCondition, MatchValue

c:\Users\ravis\AI Usecases\rbac-chatbot\rbac-chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
qdrant_store_easy = QdrantVectorStore.from_documents(
    embedding=embeddings,            # Embedding function
    path=settings.qdrant_path,           # Local persistence (optional)
    documents=[],
    collection_name="RBAC_CHATBOT" # Collection name
)

In [27]:
qdrant_store_easy.add_documents(documents=chunks)

2025-12-19 15:39:47,429 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-12-19 15:39:49,676 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"


['5cc09ed80bfe4d6ca756056b202d133b',
 '6894afefe04a4b28a5a65d8b61fc8ab7',
 'f35bea4f3d094749a21a3b8270789606',
 '1083402e33bc49b78ede5b5073054bd3',
 '18c737b3681b4fb3945f5f280f6fb26e',
 'cf55e425ceda4f6a984b5329a677cbdd',
 'bb46d8dab8d74d3bbf917321c52e5ac7',
 '24aa57f04ab24730b0682b89c7dee26d',
 '6e539f0246134c0cabdcb816fe2ea768',
 '74b49bffe38249c28909b131e4a26a13',
 'aa1094935e4a406e92ab7dc62078b026',
 '0474c3fc901d45dfb395c56b716b2abb',
 '07f03682d1e046348e72e06d1a7d8e56',
 '1a517184a21d445086d7bb60084ae304',
 '28d993939bf5473a9c66feb6cf62757d',
 '64dc2ac8512949f4aa2dbefe79c7cf80',
 '0a4424fc92444b71a2ccd40c7edfa608',
 'd749a1280db246e68063b12d86e7de50',
 'ea4429e2054d4eba96836113ee5813d9',
 '36c3b516168f4ba0ba927c547af0ef34',
 'b3b759f2bde8425dbb80e8028d09807e',
 'fe8d8ac29b9c49b7bb81f37c95112900',
 '86c50fa1e8ab4df3a071e66395b547b9',
 '8395f96130974c5d92fb84349bd43ad2',
 'c2c0ec04a1f54088bcc083a05e7f2d71',
 '9bdfd07dcf344fa7a278500c4aea65cc',
 'e286809ee1634f64b8e075ef228d684c',
 

In [10]:
allowed_roles = list(set(['engineering'] + ["general"]))
filter = Filter(
    should=[
        FieldCondition(key="metadata.role", match=MatchValue(value=role))
        for role in allowed_roles
    ]
)
qdrant_store_easy.similarity_search(query="Explain Security and Compliance",k=5,filter=filter)

[]

In [12]:
qdrant_store_easy.similarity_search(query="Leave Policy",k=5)

[]

In [15]:
retriever = qdrant_store_easy.as_retriever(search_type="similarity",
            k = 5)

In [16]:
retriever.invoke("Explain Leave Policy")

[]